In [3]:
def installRoot():
  !pip install -q condacolab

  import condacolab

  condacolab.install()

  !sed -i '/cudatoolkit/d' /usr/local/conda-meta/pinned
  !conda config --add channels conda-forge
  !conda config --set channel_priority strict
  !conda install root_base
  #!conda install pytorch torchvision -c pytorch -c conda-forge

try:
  import ROOT
except:
  print("ROOT not installed. Installing ROOT. The notebook will be restarted once the installation succeeds.")
  installRoot()

#the notebook may crash after the first run of this cell.
#When you run the cell again you should see “Welcome to JupyROOT 6.30/04”.

In [4]:
# Step 1: Download the ROOT file

!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1b6YRL54E5c0xU777nZ9yZ7cCqyDmbe7R' -O DatasetAandB_KaonTrackRefit_Bwin_new_21aug13.root

#IF you want to save any already downloaded file to your Google Drive:
#On left panel, click ''folder'' icon.
#navigate the menu and select where you want to save the file
#Right click and copy the path then use
# !cp FILE_NAME.root YOUR_PATH
##!cp Histos_Data_ZeroBias_1aprnew_goodZB_last_OK.root /content/drive/MyDrive/Course_exercises/
#Please don't forget to update the following cell according to the path here.



--2025-04-11 11:16:36--  https://docs.google.com/uc?export=download&id=1b6YRL54E5c0xU777nZ9yZ7cCqyDmbe7R
Resolving docs.google.com (docs.google.com)... 74.125.142.139, 74.125.142.113, 74.125.142.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1b6YRL54E5c0xU777nZ9yZ7cCqyDmbe7R&export=download [following]
--2025-04-11 11:16:37--  https://drive.usercontent.google.com/download?id=1b6YRL54E5c0xU777nZ9yZ7cCqyDmbe7R&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.98.132, 2607:f8b0:400e:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.98.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76239 (74K) [application/octet-stream]
Saving to: ‘DatasetAandB_KaonTrackRefit_Bwin_new_21aug13.root’

DatasetAandB_KaonTr 100%[============

In [5]:
f1 = ROOT.TFile("/content/DatasetAandB_KaonTrackRefit_Bwin_new_21aug13.root", "READ") #open only in READ mode
f1.ls()

TFile**		/content/DatasetAandB_KaonTrackRefit_Bwin_new_21aug13.root	
 TFile*		/content/DatasetAandB_KaonTrackRefit_Bwin_new_21aug13.root	
  KEY: TH2F	myjpsiphi_Vs_phikmassSq_sig;1	myjpsiphi_Vs_phikmassSq_sig
  KEY: TH2F	myjpsiphi_Vs_phikmassSq_sb;1	myjpsiphi_Vs_phikmassSq_sb
  KEY: TH1F	myJpsiKmass;1	myJpsiKmass
  KEY: TH1F	myPhiKmass;1	myPhiKmass
  KEY: TH1F	myInclusiveMuMumass;1	myInclusiveMuMumass
  KEY: TH1F	myFitMuMumassfromX;1	myFitMuMumassfromX
  KEY: TH1F	myJPsicand;1	myJPsicand
  KEY: TH1F	muonspT;1	muonspT
  KEY: TH1F	muonspT_dsA;1	muonspT_dsA
  KEY: TH1F	muonspT_dsB;1	muonspT_dsB
  KEY: TH2F	myFitMuMumass_Vs_JpsipT;1	myFitMuMumass_Vs_JpsipT
  KEY: TH1F	JpsiFLsig_h;1	JpsiFLsig_h
  KEY: TH1F	JpsiFLsig_hcut;1	JpsiFLsig_hcut
  KEY: TH1F	XVtxCL_h;1	XVtxCL_h
  KEY: TH1F	XVtxCL_hcut;1	XVtxCL_hcut
  KEY: TH1F	kaonspT_h;1	kaonspT_h
  KEY: TH1F	kaonspT_hcut;1	kaonspT_hcut
  KEY: TH1F	kaon1pT_h_sig;1	kaon1pT_h_sig
  KEY: TH1F	kaon2pT_h_sig;1	kaon2pT_h_sig
  KEY: TH1F	kaon3pT_h_sig;1	ka

In [6]:
from google.colab import drive
drive.mount('/content/drive')

import os

# Step 2: Specify the path in your Google Drive to save the data
# By default, a 'Colab Notebooks' folder is created in your Drive. You can create a subfolder if needed.
# Define the path where you want to save the data

drive_path = '/content/drive/My Drive/Alexis_exercises/'

# Create the folder if it doesn't exist
os.makedirs(drive_path, exist_ok=True)

# Define the directory path for saving plots in Google Drive
plot_dir = "/content/drive/My Drive/Colab Notebooks/Plots/"

os.makedirs(plot_dir, exist_ok=True)


Mounted at /content/drive


In [7]:
import ROOT
from ROOT import gROOT, gStyle, TCanvas, TFile, TH1D, RooRealVar, RooDataHist, RooFit, TObject,RooArgSet, RooCBShape
from ROOT import RooPlot, RooGaussian, RooChebychev, RooAddPdf, RooArgList, kRed, kGreen, kDashed, kBlue, gPad, TLine
from ROOT import RooVoigtian, kFALSE, kTRUE, RooMinimizer

In [8]:
gROOT.SetStyle("Plain")
gStyle.SetOptStat(10)
gStyle.SetTitleOffset(1.2, "")

myC = TCanvas("myC", "Plots", 1000, 800)

hist = f1.Get("myJpsiKKKmass_all")

nBins = hist.GetNbinsX()
entries = hist.GetEntries()  # this containes also overflow so I recalculate by hand as follows:
myEntries = 0
for l in range(nBins):
    myEntries += hist.GetBinContent(l + 1)

In [9]:
x = RooRealVar("x", "m(J/#psi #phi K)[GeV]", 5.15, 5.45)
Bmass = RooDataHist(hist.GetName(), hist.GetTitle(), RooArgSet(x), RooFit.Import(hist, False))

begin = 5.15
end = 5.45

xframe = x.frame()
Bmass.plotOn(xframe)

myC.cd()


In [10]:
# -- SIGNAL
mg = RooRealVar("mg", "Gaussian's mean", 5.28, 5.275, 5.285)
wg = RooRealVar("wg", "Gaussian's width", 0.010, 0.005, 0.015)
gauss1 = RooGaussian("gauss1", "Gauss(x,mg,wg)", x, mg, wg)

# -- BKG
c0 = RooRealVar("c0", "1st coeff", 0.5, -1000., 1000.)
c1 = RooRealVar("c1", "2nd coeff", -0.5, -1000., 1000.)
#--RooRealVar c2("c2","3rd coeff",0.1,-1000.,1000.); // 2nd order degree is enough here
cheby = RooChebychev("cheby", "Chebyshev", x, RooArgList(c0, c1))  # 2 coeff. means 2nd order polynominal

# -- TOTAL pdf : f*gauss1 + (1-f)*cheby
fsig = RooRealVar("fsig", "narrow fraction", 0.05, 0.0, 1.0)
model = RooAddPdf("model", "gauss1+cheby", RooArgList(gauss1, cheby), fsig)  # configured in this way this is not extended

# -- Execute FIT
# model.fitTo(Bmass,RooFit::Minos(kTRUE)); // let us give explicitely also the FitRange:
model.fitTo(Bmass, RooFit.Minos(kTRUE), RooFit.Range(begin, end))
model.plotOn(xframe, RooFit.LineColor(kRed))
model.plotOn(xframe, RooFit.Components(cheby), RooFit.LineStyle(kDashed))
model.paramOn(xframe, RooFit.Parameters(RooArgSet(mg, wg, fsig)), RooFit.Layout(0.53, 0.9, 0.9))  # 3rd is up

# double cands = fsig.getVal()*myEntries; // in case you want to use it later as a variable

print("\n # of entries =", myEntries, "of which # signal candidates is =", fsig.getVal() * myEntries, "+/-", fsig.getError() * myEntries)

xframe.SetTitle("Not extended fit : just fsig and (1-fsig)")
xframe.Draw()


 # of entries = 98475.0 of which # signal candidates is = 2921.5459581342934 +/- 174.97989432391316
[#1] INFO:Eval -- RooRealVar::setRange(x) new range named 'fit' created with bounds [5.15,5.45]
[#1] INFO:Eval -- RooRealVar::setRange(x) new range named 'fit_nll_model_myJpsiKKKmass_all' created with bounds [5.15,5.45]
[#1] INFO:Fitting -- RooAbsPdf::fitTo(model) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using CPU computation library compiled with -mavx2
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_myJpsiKKKmass_all) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 2500 convergence for edm < 1 strategy 1
RooAbsMinimizerFcn: Minimized function has error status.
Returning maximum FCN so far (-100320) to force MIGRAD to back out of this region. Error log follows.
Parameter valu

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -100320.3255 Edm =   1.960690434e+14 NCalls =     25
Info in <Minuit2>: NegativeG2LineSearch Doing a NegativeG2LineSearch since one of the G2 component is negative
Info in <Minuit2>: MnSeedGenerator Negative G2 found - new state: 
  Minimum value : -115728.4912
  Edm           : 3871.766653
  Internal parameters:	[ -0.0001726263636 -0.0005000000208     -1.119769515                0   8.67361738e-17]	
  Internal gradient  :	[     -137718.8646     -15406336.12       5201.19299      44.44397032     -60.46326491]	
  Internal covariance matrix:
[[  3.6705141e-11              0              0              0              0]
 [              0  5.6641922e-11              0              0              0]
 [              0              0  7.2370672e-05              0              0]
 [              0              0              0    0.019680186         

# Calculate a *chiSquared* (from a curve and a histogram in a **RooPlot**)

---


Note: will try two approaches:


1st approach
#
--https://root.cern.ch/doc/master/classRooPlot.html



**Syntax**: chiSquare (const char *pdfname, const char *histname, int nFitParam=0) const

**Description** : Calculate and return reduced chi-squared between a curve and a histogram.



---



**Syntax**: chiSquare (int nFitParam=0) const

**Description**: Shortcut for RooPlot::chiSquare(const char* pdfname, const char* histname, int nFitParam=nullptr)


In [11]:
xframeChi2 = x.frame()
Bmass.plotOn(xframeChi2)  # histogram (type RooDataHist)
model.plotOn(xframeChi2, RooFit.LineColor(kRed))  # curve

floatPars = model.getParameters(Bmass)
numFreeParams = floatPars.getSize()
print("\n # of free fit params is =", numFreeParams)

# normalized chiSquared is given by chi2 divided by ndof = (# bins of the fit range - #free params)
# though, the following is given already normalized!chi2Norm = xframeChi2.chiSquare(numFreeParams)

chi2Norm = xframeChi2.chiSquare(numFreeParams)
print("\n the Chi2 for the not-extended fit is =", chi2Norm)



 # of free fit params is = 5

 the Chi2 for the not-extended fit is = 1.158048100106928
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) p.d.f was fitted in a subrange and no explicit Range() and NormRange() was specified. Plotting / normalising in fit range. To override, do one of the following
	- Clear the automatic fit range attribute: <pdf>.removeStringAttribute("fitrange");
	- Explicitly specify the plotting range: Range("<rangeName>").
	- Explicitly specify where to compute the normalisation: NormRange("<rangeName>").
	The default (full) range can be denoted with Range("") / NormRange("").
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) only plotting range 'fit_nll_model_myJpsiKKKmass_all'
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) p.d.f. curve is normalized using explicit choice of ranges 'fit_nll_model_myJpsiKKKmass_all'


In [12]:
# -- 2nd approach
#
chi2 = model.createChi2(Bmass, RooFit.Range(begin, end))
# if extended  ... add Extended(true): createChi2(Bmass,Range(begin,end),Extended(true))
# because in this way the prediction of the total number of events is taken from the extended pdf and not from the histogram
print("\n chi2 (not normalized) is =", chi2.getVal())
# maybe this is the best way but be careful with the parameters you pass to the createChi2 function
# since it does not divide by ndf, i.e. the number of non-zero bins minus the number of parameters;
# you have to do this manually afterwards!
#
nDOF = nBins - numFreeParams
print("\n nDOF is =", nDOF)
chi2NormCalc = (chi2.getVal()) / nDOF
print("\n chi2 (normalized by manual calculation) is =", chi2NormCalc)



 chi2 (not normalized) is = 63.36358758325936

 nDOF is = 55

 chi2 (normalized by manual calculation) is = 1.1520652287865338


Still, this method doesn't get it right if your pdf is continuous, because it just takes the pdf value in the bin center. If this is a problem for you, you can consider wrapping the pdf in a RooBinSamplingPdf - to be explored - which turns the continuous pdf into a binned pdf where the values for each bin are obtained by integration. However, this can be important only in case of steep functions where value at the center of the bin and integral over the bin may differ; this is not the case here!

In [13]:
model.plotOn(xframeChi2, RooFit.Components(cheby), RooFit.LineStyle(kDashed))
model.paramOn(xframeChi2, RooFit.Parameters(RooArgSet(mg, wg, fsig)), RooFit.Layout(0.53, 0.9, 0.9))

myLatChi2 = ROOT.TLatex(5.35, 400., f"#chi^{{2}}={chi2NormCalc}")
xframeChi2.addObject(myLatChi2)

myLatCands = ROOT.TLatex(5.318, 2600., f"nsig={fsig.getVal() * myEntries:.1f}")
myLatCands1 = ROOT.TLatex(5.38, 2600., f"#pm{fsig.getError() * myEntries:.1f}")  # it rounds as expected
myLatCands.SetTextSize(0.038)
myLatCands1.SetTextSize(0.038)
xframeChi2.addObject(myLatCands)
xframeChi2.addObject(myLatCands1)

xframeChi2.SetTitle("Not extended fit : just fsig")
xframeChi2.SetYTitle("Candidates/(5MeV)")
xframeChi2.SetTitleOffset(1.32, "Y")
xframeChi2.Draw()

myC.SaveAs("./myBmass.png")

myC.Update()
myC.cd()


[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (cheby)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: ()


Info in <TCanvas::Print>: png file ./myBmass.png has been created


# **EXTENDED FIT**

In [14]:
myC.Divide(1, 1)

y = RooRealVar("y", "m(J/#psi #phi K)[GeV]", 5.15, 5.45)
yframe = y.frame()

BmassExt = RooDataHist(hist.GetName(), hist.GetTitle(), RooArgSet(y), RooFit.Import(hist, False))
BmassExt.plotOn(yframe)
# myC.cd()  # decomment to plot
# yframe.Draw()  # decomment to plot

mge = RooRealVar("mge", "Gaussian's mean", 5.28, 5.275, 5.285)
wge = RooRealVar("wge", "Gaussian's width", 0.01, 0.005, 0.015)
gausse = RooGaussian("gausse", "Gauss(y,mge,wge)", y, mge, wge)

c0e = RooRealVar("c0e", "1st coeff", 0.5, -1000, 1000)
c1e = RooRealVar("c1e", "2nd coeff", -0.5, -1000, 1000)
#c1e = RooRealVar("c2e","3rd coeff",-0.5,-1000,1000)

chebye = RooChebychev("chebye", "Chebyshev", y, RooArgList(c0e, c1e))

nsig = RooRealVar("nsig", "n. of signal cands", 2500., 2000., 3800.)
nbkg = RooRealVar("nbkg", "n. of bkg cands", 2000., 0., 200000.)

model_extended = RooAddPdf("model_extended", "gauss+cheby EXT", RooArgList(gausse, chebye), RooArgList(nsig, nbkg))

nll = model_extended.createNLL(BmassExt)
#nll = model_extended.createNLL(BmassExt, RooFit.NumCPU(4))

#--For Any system you can use in Python:
#import multiprocessing
#print(f"Number of CPUs on Google Colab: {multiprocessing.cpu_count()}")

m = RooMinimizer(nll)
m.migrad()



0

[#1] INFO:Minimization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Fitting -- RooAbsPdf::fitTo(model_extended) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_extended_myJpsiKKKmass_all) Summation contains a RooNLLVar, using its error level
Minuit2Minimizer: Minimize with max-calls 3000 convergence for edm < 1 strategy 1
RooAbsMinimizerFcn: Minimized function has error status.
Returning maximum FCN so far (-881401) to force MIGRAD to back out of this region. Error log follows.
Parameter values: 	c0e=130.664	c1e=-0.5	mge=5.28	nbkg=2000	nsig=2500	wge=0.01
RooAddPdf::model_extended[ nsig * gausse_over_gausse_Int[y] + nbkg * chebye_over_chebye_Int[y] ]
     getLogVal() top-level p.d.f evaluates to NaN @ !refCoefNorm=(y = 5.3), !pdfs=(gausse_over_gausse_Int[y] = 5.3991,chebye_over_chebye_Int[y] = 4.28571), !coefficients=(nsig = 2500,nbkg = 2000)
    

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -881400.8559 Edm =   7.912156014e+12 NCalls =     29
Info in <Minuit2>: NegativeG2LineSearch Doing a NegativeG2LineSearch since one of the G2 component is negative
Info in <Minuit2>: MnSeedGenerator Negative G2 found - new state: 
  Minimum value : -904246.4783
  Edm           : 105788.1094
  Internal parameters:	[  0.0005000000208  0.0003403721467                0     -1.370461484       1.57593986   8.67361738e-17]	
  Internal gradient  :	[       21084049.7     -2420957.994      666.2093529     -765677.5685      19.15200615     -13945.85639]	
  Internal covariance matrix:
[[  4.3989872e-11              0              0              0              0              0]
 [              0  5.3242719e-11              0              0              0              0]
 [              0              0   0.0033197611              0              0         

In [15]:
m.hesse()


0

Info in <Minuit2>: Minuit2Minimizer::Hesse Using max-calls 3000
Warning in <Minuit2>: MnPosDef non-positive diagonal element in covariance matrix[ 4 ] = -19.6006
Warning in <Minuit2>: MnPosDef Added to diagonal of Error matrix a value 20.1006
Info in <Minuit2>: Minuit2Minimizer::Hesse Hesse is valid - matrix is accurate


In [ ]:
# m.minos(nsig)  # get asymmetric just for parameter "nsig"


In [16]:
m.minos()  # get asymmetric for all parameters


80

******************************************************************************************************
Minuit2Minimizer::GetMinosError - Run MINOS LOWER error for parameter #0 : c0e using max-calls 3000, tolerance 1
RooAbsMinimizerFcn: Minimized function has error status.
Returning maximum FCN so far (-852104) to force MIGRAD to back out of this region. Error log follows.
Parameter values: 	c0e=-0.228667	c1e=-369.409	mge=5.27995	nbkg=101222	nsig=2039.64	wge=0.0137137
RooAddPdf::model_extended[ nsig * gausse_over_gausse_Int[y] + nbkg * chebye_over_chebye_Int[y] ]
     getLogVal() top-level p.d.f evaluates to NaN @ !refCoefNorm=(y = 5.3), !pdfs=(gausse_over_gausse_Int[y] = 9.99102,chebye_over_chebye_Int[y] = 9.9463), !coefficients=(nsig = 2039.64 +/- 1284.58,nbkg = 101222 +/- 313.857)
     getLogVal() top-level p.d.f evaluates to NaN @ !refCoefNorm=(y = 5.3), !pdfs=(gausse_over_gausse_Int[y] = 9.99102,chebye_over_chebye_Int[y] = 9.9463), !coefficients=(nsig = 2039.64 +/- 1284.58,nbkg = 1

Info in <Minuit2>: MnMinos Determination of lower Minos error for parameter 0
Info in <Minuit2>: MnFunctionCross Run Migrad with fixed parameters:
  Pos 0: c0e = -0.228667
Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -1153528.691 Edm =      0.1113564963 NCalls =     13
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : -1153528.691
  Edm           : 0.1113564963
  Internal parameters:	[  2.519547981e-06    -0.1153322811   -0.05242768349      1.553585924     0.3356480039]	
  Internal gradient  :	[      3768.859858    -0.1852415426    -0.5505295638      0.331637999   -0.01818935393]	
  Internal covariance matrix:
[[  6.0465212e-11  4.8012674e-08 -1.2621067e-09 -9.3653589e-07 -4.6853872e-08]
 [  4.8012674e-08    0.020427494 -4.5698621e-07 -0.00033905252 -9.6316726e-05]
 [ -1.2621067e-09 -4.5698621e-07  1.9754631e-05   0.0005599142  3.8598664e-05]
 [ -9.3653589e-07 -0.0003

In [ ]:
nsig.Print()


RooRealVar::nsig = 2923.26 +/- (-174.295,176.431)  L(2000 - 3800) 


In [17]:
floatParsExt = model_extended.getParameters(BmassExt)
numFreeParamsExt = floatParsExt.getSize()
print("\n # of free extended-fit params is =", numFreeParamsExt)

nDOFExt = nBins - numFreeParamsExt
print("\n nDOF is =", nDOFExt)
chi2Ext = model_extended.createChi2(BmassExt, RooFit.Range(begin, end))
chi2NormExtCalc = (chi2Ext.getVal()) / nDOFExt
print("\n chi2 (normalized by manual calculation) is =", chi2NormExtCalc)


 # of free extended-fit params is = 6

 nDOF is = 54

 chi2 (normalized by manual calculation) is = 1.1734199611826555
[#1] INFO:Minimization -- p.d.f. provides expected number of events, including extended term in likelihood.


In [18]:
fitres = m.save()
gStyle.SetPalette(1)  # for better color choice
fitres.correlationHist().Draw("colz")
myC.SaveAs("./myCorrelationMatrix.png")
myC.Update()
myC.cd()

# notice that - as expected - gaussian width (wge) is correlated with the signal yield (nsig)
# and also the latter is anticorrelated with the background yield

Info in <TCanvas::Print>: png file ./myCorrelationMatrix.png has been created


In [19]:
model_extended.plotOn(yframe, RooFit.LineColor(kRed))
model_extended.plotOn(yframe, RooFit.Components(chebye), RooFit.LineStyle(kDashed))
model_extended.paramOn(yframe, RooFit.Parameters(RooArgSet(mge, wge, nsig, nbkg)), RooFit.Layout(0.53, 0.9, 0.9))
yframe.SetTitle("Extended fit : nsig and nbkg")

myLatExt = ROOT.TLatex(5.35, 400., f"#chi^{{2}}={chi2NormExtCalc}")
yframe.addObject(myLatExt)
yframe.SetYTitle("Candidates/(5MeV)")
yframe.SetTitleOffset(1.32, "Y")
yframe.Draw()

myC.SaveAs("./myBmassExtended.png")
myC.Update()
myC.cd()

[#1] INFO:Plotting -- RooAbsPdf::plotOn(model_extended) directly selected PDF components: (chebye)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model_extended) indirectly selected PDF components: ()


Info in <TCanvas::Print>: png file ./myBmassExtended.png has been created


In [20]:
# fit is done but now we want to derive more info from the RooFitResult object
model_extended.plotOn(yframe, RooFit.VisualizeError(fitres))
yframe.Draw()
BmassExt.plotOn(yframe)
yframe.Draw("Esame")

myC.SaveAs("./myErrorVisualization.png")
myC.Update()
myC.cd()

Info in <TCanvas::Print>: png file ./myErrorVisualization.png has been created


In [21]:
# The ''contour'' method is the only way you can add 1, 2 and 3 sigma contours to a RooPlot object.
# Check: https://root.cern/doc/v606/RooMinimizer_8cxx_source.html#l00698

contourFrame = m.contour(nsig, wge, 1, 2, 3, 0, 0, 0)  # gives the 3 contours obtained for 1, 2 and 3 sigmas
#If you want to and more you can draw up to 6, so (nsig, wgen, 1,2,3,4,5,6)
contourFrame.SetTitle("Minuit contour plots at 1,2 and 3 sigmas")
contourFrame.SetTitleOffset(1.38, "Y")
contourFrame.Draw()
myC.SaveAs("./paramContours_nsig_wge.png")
myC.Update()
myC.cd()

Info in <Minuit2>: Minuit2Minimizer::Contour Computing contours - 0.5
Info in <Minuit2>: Minuit2Minimizer::Contour Computing contours - 2
Info in <Minuit2>: Minuit2Minimizer::Contour Computing contours - 4.5
Info in <TCanvas::Print>: png file ./paramContours_nsig_wge.png has been created


In [22]:
hasattr(fitres, "createHessePdf")


True

In [23]:
paramPDF = fitres.createHessePdf(ROOT.RooArgSet(nsig, wge))
paramPDF.Print()


RooMultiVarGaussian::pdf_RooEvaluatorWrapper[ x=(nsig,wge) mu=(nsig_centralvalue,wge_centralvalue) ] = 1


In [24]:
# now plot Likelihood and Profile Likelihood Ratio functions

myC.Divide(1, 1)

# plot the likelihood as a function of the parameter of interest (here nsig):
nsig_frame = nsig.frame(RooFit.Bins(60), RooFit.Range(2000, 3800))
nll.plotOn(nsig_frame, RooFit.ShiftToZero(), RooFit.LineStyle(ROOT.kDashed), RooFit.LineColor(ROOT.kBlue))

# make the Profile Likelihood ratio (that can be represented as a regular RooFit function)
pll_nsig = nll.createProfile(nsig)
pll_nsig.plotOn(nsig_frame, RooFit.ShiftToZero(), RooFit.LineColor(ROOT.kRed))

nsig_frame.SetMinimum(-1)
nsig_frame.SetMaximum(5)
nsig_frame.Draw()

line0 = ROOT.TLine(2000, 0, 3800, 0)
line0.SetLineColor(1)
line0.SetLineWidth(2)
line0.SetLineStyle(2)
line0.Draw("same")

line05 = ROOT.TLine(2000, 0.5, 3800, 0.5)
line05.SetLineColor(1)
line05.SetLineWidth(2)
line05.SetLineStyle(2)
line05.Draw("same")

lineN1 = ROOT.TLine(3100, -1., 3100, 0.5)
lineN1.SetLineColor(2)
lineN1.SetLineWidth(1)
lineN1.SetLineStyle(2)
lineN1.Draw("same")

lineN2 = ROOT.TLine(2749, -1., 2749, 0.5)
lineN2.SetLineColor(2)
lineN2.SetLineWidth(1)
lineN2.SetLineStyle(2)
lineN2.Draw("same")

myC.SaveAs("./myLikelihood.png")
myC.Update()
myC.cd()

del myC

ROOT.gROOT.Reset()
ROOT.gROOT.Clear()


[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[nsig]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_extended_myJpsiKKKmass_all) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[nsig]) determining minimum likelihood for current configurations w.r.t all observable
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[nsig]) minimum found at (nsig=2923.34)
..........................................................................................................................

Info in <TCanvas::Print>: png file ./myLikelihood.png has been created


[#1] INFO:Minimization -- RooProfileLL::evaluate(nll_model_extended_myJpsiKKKmass_all_Profile[nsig]) Creating instance of MINUIT
[#1] INFO:Minimization -- RooProfileLL::evaluate(nll_model_extended_myJpsiKKKmass_all_Profile[nsig]) determining minimum likelihood for current configurations w.r.t all observable
[#1] INFO:Minimization -- RooProfileLL::evaluate(nll_model_extended_myJpsiKKKmass_all_Profile[nsig]) minimum found at (nsig=2919.07)
..........................................................................................................................

Info in <TCanvas::Print>: png file ./myLikelihood.png has been created
